In [1]:
import pandas as pd
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_absolute_error

from analytics.machine_learning.price_prediction_with_fundamentals import utils

In [2]:
dataset = utils.get_dataset()

In [3]:
import datetime as dt

train_set, test_set = utils.split_data_to_train_and_test(
    df=dataset,
    cutoff_date=dt.datetime(2023,6,1)
)

In [4]:
y_train = train_set[['price', 'sector']]
X_train = train_set.drop(['price'], axis=1)

y_test = test_set[['price', 'sector']]
X_test = test_set.drop(['price'], axis=1)

In [5]:
from sklearn.preprocessing import (
    OneHotEncoder,
    MinMaxScaler
)

one_hot_encoder = OneHotEncoder()
scaler = MinMaxScaler()

X_train_transformed = utils.transform_input(
    X=X_train,
    one_hot_encoder=one_hot_encoder,
    scaler=scaler,
    fit=True
)

X_test_transformed = utils.transform_input(
    X=X_test,
    one_hot_encoder=one_hot_encoder,
    scaler=scaler,
    fit=False
)

In [19]:
from tensorflow import keras

# Build the neural network model
model = keras.Sequential([
    # Input layer
    keras.layers.Input(shape=(100,)),  # Adjust the input shape according to your data
    
    # Hidden layers
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    
    # Output layer (a single neuron for regression)
    keras.layers.Dense(1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_absolute_error')  # You can also use 'mean_absolute_error' for MAE

# Train the model
history = model.fit(X_train_transformed, y_train['price'], epochs=100, batch_size=32, validation_data=(X_test_transformed, y_test['price']))

# Evaluate the model on the test set
loss = model.evaluate(X_test_transformed, y_test['price'])
print(f"Test Loss: {loss}")

# You can also make predictions using the trained model
y_pred = model.predict(X_test_transformed)

# Save and load the model (optional)
model.save("regression_model.h5")
# To load the model later: loaded_model = keras.models.load_model("regression_model.h5")


Epoch 1/100
1686/1686 [==============================] - 1s 623us/step - loss: 50.3641 - val_loss: 53.0035
Epoch 2/100
1686/1686 [==============================] - 1s 560us/step - loss: 49.2187 - val_loss: 53.3269
Epoch 3/100
1686/1686 [==============================] - 1s 570us/step - loss: 48.9024 - val_loss: 51.7828
Epoch 4/100
1686/1686 [==============================] - 1s 552us/step - loss: 48.6864 - val_loss: 51.2192
Epoch 5/100
1686/1686 [==============================] - 1s 588us/step - loss: 48.4000 - val_loss: 51.2604
Epoch 6/100
1686/1686 [==============================] - 1s 554us/step - loss: 48.1057 - val_loss: 50.5191
Epoch 7/100
1686/1686 [==============================] - 1s 569us/step - loss: 47.7139 - val_loss: 51.4771
Epoch 8/100
1686/1686 [==============================] - 1s 554us/step - loss: 47.4236 - val_loss: 52.3624
Epoch 9/100
1686/1686 [==============================] - 1s 563us/step - loss: 47.2065 - val_loss: 51.9938
Epoch 10/100
1686/1686 [=============

/Users/orestis/MyProjects/InvestorAPI/env/lib/python3.11/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [20]:
# The mean squared error
print("Mean absolute error: %.2f" % mean_absolute_error(y_test['price'], y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test['price'], y_pred))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['price'], y_pred))

Mean absolute error: 58.62
Coefficient of determination: -0.11
Mean absolute pct error: 0.86


## Try to improve it

In [6]:
from tensorflow import keras
from sklearn.metrics import mean_squared_error


# Build a more complex neural network
model = keras.Sequential([
    keras.layers.Input(shape=(100,)),
    keras.layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(128, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(1)
])

# Implement learning rate scheduling
lr_schedule = keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-2, decay_steps=10000, decay_rate=0.9)
optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

# Compile the model
model.compile(optimizer=optimizer, loss='mean_squared_error')

# Implement early stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model
history = model.fit(
    X_train_transformed,
    y_train['price'],
    epochs=100, batch_size=64,
    validation_data=(X_test_transformed, y_test['price']),
    callbacks=[early_stopping]
)

# Evaluate the model
y_pred = model.predict(X_test_transformed)
test_loss = mean_squared_error(y_test['price'], y_pred)
print(f"Test Mean Squared Error: {test_loss:.2f}")


Epoch 1/100
843/843 [==============================] - 2s 2ms/step - loss: 21882.2012 - val_loss: 32248.4531
Epoch 2/100
843/843 [==============================] - 1s 1ms/step - loss: 21389.5273 - val_loss: 28528.8496
Epoch 3/100
843/843 [==============================] - 1s 1ms/step - loss: 21050.5742 - val_loss: 47492.0117
Epoch 4/100
843/843 [==============================] - 1s 1ms/step - loss: 20676.7734 - val_loss: 63153.0352
Epoch 5/100
843/843 [==============================] - 1s 1ms/step - loss: 20046.1348 - val_loss: 114784.3750
Epoch 6/100
843/843 [==============================] - 1s 1ms/step - loss: 19804.8203 - val_loss: 74047.6016
Epoch 7/100
843/843 [==============================] - 1s 1ms/step - loss: 19668.3086 - val_loss: 570282.5625
Epoch 8/100
843/843 [==============================] - 1s 1ms/step - loss: 19559.7656 - val_loss: 4082599.5000
Epoch 9/100
843/843 [==============================] - 1s 1ms/step - loss: 19517.3242 - val_loss: 924453.1875
Epoch 10/100
8

In [7]:
# The mean squared error
print("Mean absolute error: %.2f" % mean_absolute_error(y_test['price'], y_pred))
# The coefficient of determination: 1 is perfect prediction
print("Coefficient of determination: %.2f" % r2_score(y_test['price'], y_pred))
print("Mean absolute pct error: %.2f" % mean_absolute_percentage_error(y_test['price'], y_pred))

Mean absolute error: 57.53
Coefficient of determination: -0.01
Mean absolute pct error: 3.49
